#**Demo: Building a Text Generation Pipeline with LangChain and Hugging Face's Flan T5 Large Model**

In this demo, you will learn how to create a Langchain HuggingFacePipeline for efficient text generation and dive into the creation of a Langchain chain to craft context-aware responses using a custom template.

##**Steps to Perform:**
1. Install the Required Libraries and Dependencies
2. Authenticate the Hugging Face Account and Set the API Key
3. Use the Hugging Face Hub to Load the Flan T5 Large model
4. Create a Langchain HuggingFacePipeline for Text Generation
5. Build a Chain Using Langchain
6. Test and Run the Chain on Few a Questions

###**Step 1: Install the Required Libraries and Dependencies**


*   Install the necessary libraries, including **Langchain**, **Transformers**, and **Torch**.




In [1]:
# !pip install langchain transformers torch accelerate

# **Step 2: Authenticate the Hugging Face Account and Set the API Key**

*   Click this link: https://huggingface.co/settings/tokens
*   Login or create an account.
*   Click on **New token**.
*   On the dialog box, give a name and select the role as **write**.
*   Copy the access token or API key.
*   Replace **Your_HF_Key** with the copied key.

In [2]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "os.getenv("HF_TOKEN")"

###**Step 3: Use the Hugging Face Hub to Load the Flan T5 Large model**

In [3]:
import torch
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain import HuggingFaceHub

# from langchain.llms import OpenAI

# gpt = OpenAI(openai_api_key = "Your Key")

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":1, "max_length":50})


2024-08-11 16:28:03.665211: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 16:28:03.715454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 16:28:06.273940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-11 16:28:06.282941: I tens

VOC-NOTICE: GPU memory for this assignment is capped at 2048MiB


/usr/local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


###**Step 4: Create a Langchain HuggingFacePipeline for Text Generation**

In [4]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Prompt Template

In [5]:
prompt_template_name=PromptTemplate(input_variables=["currency"],
                                    template="can you tell me the country to which this currency {currency} belongs to?")


In [6]:
# prompt = prompt_template_name.format(currency="yen")

In [7]:
# prompt

# Creating Chain

In [8]:
llm_chain1 = LLMChain(prompt = prompt_template_name, llm=llm)

###**Step 6: Test and Run the Chain on Few a Questions**

---



In [9]:
# Question 1
question = "euro"

llm_chain1.run(question)

'Spain'

In [10]:
template = """Question: {question}

Answer: Let's think step by step. Finish the answer in 25 words"""

prompt_template_name1 = PromptTemplate(template=template, input_variables=["in 25 words answer the question"])

In [11]:
llm_chain2 = LLMChain(prompt=prompt_template_name1, llm=llm)

In [12]:
# Question 2
question = "explain, What are the main causes of climate change, and how can we address them?"

llm_chain2.run(question)

'Climate change is caused by changes in the atmosphere. Climate change is caused by changes in the atmosphere. Climate change is caused by changes in the atmosphere.'

In [13]:
# Question 3
question = "Provide a brief overview of the history of artificial intelligence."

llm_chain2.run(question)

'Artificial intelligence has been around for a long time. The first computer programs were written in the 1940s. The first computer programs were written in the 1950s.'

In [14]:
prompt_template_name2 = PromptTemplate(input_variables=['cuisine'],
                                 template = "i want to open a restaurant for {cuisine} food, suggest a fancy name for this restaurant")

In [15]:
prompt_template_name2

PromptTemplate(input_variables=['cuisine'], template='i want to open a restaurant for {cuisine} food, suggest a fancy name for this restaurant')

In [16]:
llm_chain3=LLMChain(llm=llm, prompt=prompt_template_name2)

In [17]:
llm_chain3.run("american")

'The American'

In [18]:
# First Template
prompt_template_name = PromptTemplate(input_variables=["industry"],
                                    template = "I want to start a startup for {industry} , suggest me a good name for this")

name_chain = LLMChain(llm = llm, prompt = prompt_template_name)

# Second Template
prompt_template_items = PromptTemplate( input_variables = ["name"],
                                     template = "suggest 3 business strategies in bullet points for {name}")

strategies_chain = LLMChain(llm = llm, prompt = prompt_template_items)

In [19]:
from langchain.chains import SimpleSequentialChain

In [20]:
chain = SimpleSequentialChain(chains=[name_chain, strategies_chain]) # This gives us one output

In [21]:
print(chain.run("logistics"))

- Invest in a logistics startup - Invest in a logistics startup


In [22]:
# Prompt 1
prompt_template_name=PromptTemplate(input_variables=["cuisine"],
                                    template="i want to open a restaurant for this {cuisine}, suggest a fancy name for it")

name_chain=LLMChain(llm=llm, prompt=prompt_template_name,output_key="restaurant_name")
 
# Prompt 2
prompt_templates_items=PromptTemplate(input_variables=["restaurant_name"],
                                      template="suggest 3 fancy and unique menu items for this {restaurant_name}")
food_items_chain=LLMChain(llm=llm,prompt=prompt_templates_items, output_key="menu_items")

In [23]:
from langchain.chains import SequentialChain

In [24]:
chain=SequentialChain(
    chains=[name_chain, food_items_chain], input_variables=["cuisine"],
    output_variables=["restaurant_name", "menu_items"]
)

In [25]:
chain({"cuisine": "mexican"})

{'cuisine': 'mexican', 'restaurant_name': 'Mexican', 'menu_items': 'guacamole'}

In [26]:
# Now using Chain i have connect multiple thing